# Домашняя работа 

## "Наивный байесовский классификатор"

### Выполнила студентка группы Т12О-101М-20
### Давыдова Елизавета

In [29]:
from typing import List
from collections import defaultdict

import numpy as np

SPAM_CLASS = 'spam'
NOT_SPAM_CLASS = 'ham'

In [30]:
"""имплементация наивного байесовского классификатора"""
class NaiveBayes:
    def __init__(self):
        
        self.labels = [NOT_SPAM_CLASS, SPAM_CLASS]
        self.class_labels_proba = None  # априорная вероятность класса, словарь
        self.prior_word_proba = None  # частоты фичей (токенов)
    
    
    def _set_labels_prior_proba(self, target: list, data: list) -> None:
        """Вычисление априорной вероятности классов
        
        Вызов функции должен инициализировать массив self.class_labels_proba
        
        """
        class_labels_proba = dict.fromkeys(self.labels, 0.0)
        class_labels_freq = dict.fromkeys(self.labels, 0)
        num_data = len(data)
        
        for label in target:
            class_labels_proba[label] += 1
            class_labels_freq[label] +=1
            
        for label in class_labels_proba:
            class_labels_proba[label] /= num_data
            
        self.class_labels_proba = class_labels_proba
        self.class_labels_freq = class_labels_freq
        
        
    def _set_word_prior_proba(self, target, data):
        """Вычисляем априорную вероятность токенов в классе
        
        Заполняем словарь self.prior_word_proba[label][token]
        
        """
        word_proba_dict_by_class = dict.fromkeys(self.labels)
        
        for label in self.labels:
            word_proba_dict_by_class[label] = defaultdict(lambda:0)
        
        for label, tokens in zip(target, data):
            for token in tokens:
                word_proba_dict_by_class[label][token] += 1
                
        for label, tokens in word_proba_dict_by_class.items():
            for token in tokens:
                word_proba_dict_by_class[label][token] /= self.class_labels_freq [label]
                
        self.prior_word_proba = word_proba_dict_by_class
        
        
    def _tokenize_text(self, text) -> list:
        """Функция, которая разобьёт входной текст на токены
        
        Токены вернуть в виде списка"""
        
        tokens = []
        #преобразуем текст
        text_no_punctuation = ''.join([
            char
            for char in text
            if char not in string.punctuation
        ])
        text_lowercase = ' '.join([
            word.lower()
            for word in text_no_punctuation.split(sep=' ')
        ])
        #преобразованный текст разбиваем на токены
        tokens = text_lowercase.split(' ')
        return tokens
    
    def fit(self, target: list, data: list):
        """Обучение статистик по датасету

        :param data: массив документов, каждый документ - объект типа str
        :param target: массив меток объектов
        :return:
        """
        if not isinstance(data, list):
            raise ValueError('Аргумент data должен иметь тип list')
        if not isinstance(target, list):
            raise ValueError('Аргумент target должен иметь тип list')
        print('Данные инициализированы!')
        self._set_labels_prior_proba(target, data)
        print(f'Априорные вероятности классов {self.class_labels_proba}')
        self._set_word_prior_proba(target, data)
        print('Обучили априорные вероятности слов') 
        
    def _predict_proba(self, data: list) -> List[tuple]:
        """Предсказываем класс для текстовой смс

        :param data: массив документов, для каждого из которых нужно предсказать метку
        :return: вероятности для каждого из классов
        """
        prediction = []
        for obj in data:
            posterior_class_proba = defaultdict(lambda: 1)
            for token in self._tokenize_text(obj):
                for label in self.labels:
                    posterior_class_proba[label] *= self.prior_word_proba[label][token]
            # сохраняем для каждой метки класса - сколько меток, таков и размер uple
            prediction.append(
                tuple(
                    posterior_class_proba[label] for label in self.labels
                )
            )
        print(f'proba: {prediction}')
        return prediction
    
    def predict(self, data) -> List[str]:
        predict_labels = []
        for proba in self._predict_proba(data):
            predict_labels.append(self.labels[np.argmax(proba)])
        return predict_labels

    
            
            

Подготвим данные для обучения классификатора

In [31]:
import pandas as pd

filename = 'C:/Users/Лиза/Downloads/sms_spam_collection.tar.gz'

df = pd.read_csv(
    filename,
    compression='gzip',
    header=1,
    sep='\t',
    encoding='utf8',
    names=['class', 'sms_text'],
    error_bad_lines=False
)


df.head(3)

,class,sms_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,U dun say so early hor... U c already then say...
2,ham,"Nah I don't think he goes to usf, he lives aro..."


In [32]:
num_objects, num_features = df.shape
print(num_objects, num_features)

5571 2


Если есть, удалим объекты NaN

In [33]:
df = df.dropna()

Очистим данные и токенизируем текст

In [34]:
import string

def text_preprocess(sms_text: str) -> str:
    """Преобразование текста для анализа"""
    text_no_punctuation = ''.join([
        char
        for char in sms_text
        if char not in string.punctuation
    ])
    text_lowercase = ' '.join([
        word.lower()
        for word in text_no_punctuation.split(sep=' ')
    ])
    
    return text_lowercase

def tokenize_text(text: str):
    tokens = []
    processed_text = text_preprocess(text)
    tokens = text.split(' ')
    return tokens

Данные для обучения классификатора

In [35]:
all_text = df['sms_text'].tolist()
data = []

In [36]:
for s in all_text:
    data.append(tokenize_text(s))

In [37]:
targets = df['class'].tolist()

In [38]:
naive_bayes = NaiveBayes()

Обучаем классификатор

In [39]:
naive_bayes.fit(targets, data)

Данные инициализированы!
Априорные вероятности классов {'ham': 0.8658886894075404, 'spam': 0.1341113105924596}
Обучили априорные вероятности слов


In [40]:
naive_bayes.prior_word_proba['ham']['thank'], naive_bayes.prior_word_proba['spam']['thank']

(0.0018660584698320546, 0)

Предсказание метки класса

In [41]:
import numpy as np
# рандомный объект датасета

random_obj_ind = np.random.randint(low=0, high=num_objects, size=3)
random_obj_list = df['sms_text'].values[random_obj_ind].tolist()
random_target = df['class'][random_obj_ind].tolist()

print(random_obj_list)
naive_bayes.predict(
    random_obj_list
)

["I don't know u and u don't know me. Send CHAT to 86688 now and let's find each other! Only 150p/Msg rcvd. HG/Suite342/2Lands/Row/W1J6HL LDN. 18 years or over.", 'Po de :-):):-):-):-). No need job aha.', "I can ask around but there's not a lot in terms of mids up here"]
proba: [(0.0, 0.0), (9.013651262025389e-17, 0.0), (5.381465421966776e-27, 0.0)]


['ham', 'ham', 'ham']